# Enzyme Design 🥸 🔧 💊

## Problem
Use the datasets to analyze, sanitize, and design new alpha-amylase variants with improved activity. You can train or use neural networks then provide your 100 best-scoring sequences.

In [2]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.3 MB/s eta 0:00:00


In [ ]:
!pip install transformers datasets

In [ ]:
#Check if it is installed
import transformers
print(f"Transformers version: {transformers.__version__}")

Transformers version: 4.44.2


## Mount with google derive

In [1]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Cleaning
I removed the repeated sequences, those containing invalid amino acids, and sequences with less than 30% identity compared to the first sequence in the provided CSV file, using sequence alignment to verify this. To confirm the validity of removing these sequences, I used ESMFold to check their structures and compared them with other sequences. Since their structures differed significantly, I deleted them. I will address the missing values with MERGE after analyzing the protein fitness landscape.

In [4]:
import csv
from Bio import pairwise2
import pandas as pd
import re
import subprocess
import os
import numpy as np
from Bio import SeqIO
from Bio.Align import MultipleSeqAlignment
from io import StringIO
from tqdm import tqdm

# Function to calculate identity percentage between two sequences
def calculate_identity(seq1, seq2):
    alignments = pairwise2.align.globalxx(seq1, seq2)
    best_alignment = alignments[0]
    aligned_seq1 = best_alignment[0]
    aligned_seq2 = best_alignment[1]

    # Count the number of matches
    matches = sum(res1 == res2 for res1, res2 in zip(aligned_seq1, aligned_seq2))

    # Calculate identity percentage
    identity_percentage = (matches / len(aligned_seq1)) * 100
    return identity_percentage


input_file = "/content/drive/MyDrive/sequencesID.csv"
output_file = "/content/drive/MyDrive/filtered_sequences.csv"

df = pd.read_csv(input_file)

def is_valid_sequence(seq):
    # Check if the sequence contains only valid amino acid characters (A-Z, no gaps or special chars)
    return bool(re.match("^[ACDEFGHIKLMNPQRSTVWY]+$", seq))

df['valid_sequence'] = df['mutated_sequence'].apply(is_valid_sequence)
df_sanitized = df[df['valid_sequence'] == True]

df_sanitized = df_sanitized.drop_duplicates(subset=['mutated_sequence'])

df_sanitized = df_sanitized.drop(columns=['valid_sequence'])
print(df_sanitized.head())

# The first sequence is the reference
first_sequence = df_sanitized.iloc[0]['mutated_sequence']

# Filter sequences based on identity percentage (threshold = 30%)
threshold = 30
filtered_sequences = []
d = 1
for index, row in df_sanitized.iterrows():
    current_sequence = row['mutated_sequence']
    identity = calculate_identity(first_sequence, current_sequence)

    # Add the identity percentage to the current row in the DataFrame
    df_sanitized.at[index, 'Identity_Percentage'] = f"{identity:.2f}"

    print(f"Identity with {row['ID']}: {identity:.2f}%")

    if identity >= threshold:
        filtered_sequences.append(row.to_dict())
    else:
        d+=1

with open(output_file, 'w', newline='') as csvfile:
    fieldnames = ['ID', 'mutated_sequence', 'activity_dp7']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(filtered_sequences)

print(f"Filtered sequences saved to {output_file}")
print(f"Number of sequences removed: {d}")


/usr/local/lib/python3.10/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


   ID                                   mutated_sequence  activity_dp7
0   1  ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVK...     10.552655
1   2  ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVK...      9.338325
2   3  ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVK...      8.289414
3   4  ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVK...      7.007689
4   5  ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVK...      6.898567
Identity with 1: 100.00%
Identity with 2: 98.60%
Identity with 3: 99.06%
Identity with 4: 98.14%
Identity with 5: 99.06%
Identity with 6: 99.53%
Identity with 7: 98.14%
Identity with 8: 99.06%
Identity with 9: 97.22%
Identity with 10: 98.14%
Identity with 11: 98.60%
Identity with 12: 98.60%
Identity with 13: 99.06%
Identity with 14: 98.14%
Identity with 15: 97.67%
Identity with 16: 98.60%
Identity with 17: 96.76%
Identity with 18: 97.67%
Identity with 19: 98.60%
Identity with 20: 98.60%
Identity with 21: 97.67%
Identity with 22: 98.14%
Identity with 23: 98.60%
Identit

After executing this step, 21 sequences were removed from the dataset.

## CSV_2_FASTA
I needed a FASTA file for the next steps of the code, so I generate it.




In [ ]:

import pandas as pd


csv_file = '/content/drive/MyDrive/filtered_sequences.csv'
data = pd.read_csv(csv_file)
data = data.dropna(subset=['mutated_sequence'])


ids = data['ID']
sequences = data['mutated_sequence']

fasta_file_path = '/content/drive/MyDrive/sequences.fasta'
with open(fasta_file_path, 'w') as fasta_file:
    for index, (id_, sequence) in enumerate(zip(ids, sequences)):
        fasta_file.write(f'>{id_}\n')
        fasta_file.write(f'{sequence}\n')

print(f"FASTA file created at: {fasta_file_path}")


FASTA file created at: /content/drive/MyDrive/sequences.fasta


# Approaches to Address This Problem:
## Methods for Generating Sequences:
These are three possible approaches to address these problems. 
1) Using [ZymCTRL](https://huggingface.co/AI4PD/ZymCTRL) (Enzyme Control) which is a conditional language model for the generation of artificial functional enzymes.

2) Using ESM to obtain per-residue representations of the enzyme, introducing noise to the unconserved positions, and then mapping back to sequence space.

3) Identifying the active site and generating new sequences using ProteinMPNN by leveraging its structure-based design capabilities.

4) Tracking the changes!

## Scoring Approach:
To score the generated sequences, we can use three approches:

1) Using ESM to find protein representations and applying PCA to reduce their dimensionality, resulting in a smaller representation. The reduced representation is then used for Bayesian learning to label the generated sequences. PCA is effective here because most residues are conserved, allowing for a more compact feature set suitable for Bayesian learning.

2) Using pretrained model like [MERGE](https://github.com/amillig/MERGE), which is capable of identifying the protein fitness landscape, even with a limited amount of data. Additionally, it is possible to generate new sequences with MERGE after determining the fitness landscape.

3) Training an encoder, decoder, or fine-tuning one, on the generated sequences allows us to extract meaningful representations from those sequences. The representation should be compact, and since we are working with only one family, it is likely possible to obtain a smaller representation than what current deep learning models typically produce. Once we have these representations, we can perform Bayesian learning on the labeled data, and subsequently apply the trained model to the unlabeled data.


In [ ]:
!git lfs install

Git LFS initialized.


In [ ]:
!git clone https://huggingface.co/AI4PD/ZymCTRL

fatal: destination path 'ZymCTRL' already exists and is not an empty directory.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
from Bio import SeqIO
import numpy as np

model_path = '/content/ZymCTRL'

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(458, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=458, bias=False)
)

## 1. Using ZymCTRL
There are two ways to use ZymCTRL: the first is to define the enzyme family you want to generate, and the second is to fine-tune the network with your own data for more accurate sequences. In this case, I used the second approach and fine-tuned the model with the dataset from the CSV file. The code I used could be found in their [huggingface](https://huggingface.co/AI4PD/ZymCTRL) .

In [ ]:
import random
from transformers import AutoTokenizer

from datasets import load_dataset
import transformers
from transformers.testing_utils import CaptureLogger

## DEFINE THESE VARIABLES
tokenizer = AutoTokenizer.from_pretrained('AI4PD/ZymCTRL')
ec_label = '3.2.1.1' # CHANGE TO YOUR LABEL
validation_split_percentage = 10 # change if you want
sequence_file = '/content/drive/MyDrive/sequences.fasta'


#Load sequences, Read source file
with open(sequence_file, 'r') as fn: #! CHANGE TO SEQUENCES.FASTA
    data = fn.readlines()
    fn.close()

# Put sequences into dictionary
sequences={}
for line in data:
    if '>' in line:
        name = line.strip()
        sequences[name] = []  #! CHANGE TO corre
        continue
    sequences[name].append(line.strip())

#Pass sequences to list and shuffle their order randomly
sequences_list = [(key,value[0]) for key,value in sequences.items()]
random.shuffle(sequences_list)

#the objective is to get here strings, that when tokenized, would span a length of 1024.
#for each sequence group its length and untokenized string
print("procesing dataset")
processed_dataset = []
for i in sequences_list:
    # length of the control code
    sequence = i[1].strip()
    separator = '<sep>'
    control_code_length = len(tokenizer(ec_label+separator)['input_ids'])
    available_space = 1021 - control_code_length # It is not 1024 because '<|endoftext|>', and start and end

    # Option 1: the sequence is larger than the available space (3-4% of sequences)
    if len(sequence) > available_space:
        total_length = control_code_length + len(sequence[:available_space]) + 1
        seq = f"{ec_label}{separator}{sequence[:available_space]}<|endoftext|>"
        processed_dataset.append((total_length, seq))

    # Option 2 & 3: The sequence fits in the block_size space with or without padding
    else:
        total_length = control_code_length + len(sequence) + 3
        # in this case the sequence does not fit with the start/end tokens
        seq = f"{ec_label}{separator}<start>{sequence}<end><|endoftext|>"
        processed_dataset.append((total_length, seq))

# Group sequences
def grouper(iterable):
    prev = None
    group = ''
    total_sum = 0
    for item in iterable:
        if prev is None or item[0] + total_sum < 1025:
            group += item[1]
            total_sum += item[0]
        else:
            total_sum = item[0]
            yield group
            group = item[1]
        prev = item
    if group:
        total_sum = 0
        yield group

print("grouping processed dataset")
grouped_dataset=dict(enumerate(grouper(processed_dataset),1))

# Write file out for the tokenizer to read
fn = open(f"{ec_label}_processed.txt",'w')
for key,value in grouped_dataset.items():
    padding_len = 1024 - len(tokenizer(value)['input_ids'])
    padding = "<pad>"*padding_len
    fn.write(value+padding)
    fn.write
    fn.write("\n")
fn.close()

##TOKENIZE
# adapted from the trainer file
data_files = {}
dataset_args = {}

data_files["train"] = f"{ec_label}_processed.txt"
extension = "text"
tok_logger = transformers.utils.logging.get_logger("transformers.tokenization_utils_base")

raw_datasets = load_dataset(extension, data_files=data_files, cache_dir='.', **dataset_args)

raw_datasets["train"] = load_dataset(extension,
                data_files=data_files,
                split=f"train[{validation_split_percentage}%:]",
                cache_dir='.',
                **dataset_args,)

raw_datasets["validation"] = load_dataset(extension,
                                          data_files=data_files,
                                          split=f"train[:{validation_split_percentage}%]",
                                          cache_dir='.',
                                          **dataset_args,)

def tokenize_function(examples):
    with CaptureLogger(tok_logger) as cl:
        output = tokenizer(examples["text"])
    # clm input could be much much longer than block_size
    if "Token indices sequence length is longer than the" in cl.out:
        tok_logger.warning(
            "^^^^^^^^^^^^^^^^ Please ignore the warning above - this long input will be chunked into smaller bits before being passed to the model."
        )
    return output

tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    num_proc=32,
    remove_columns=['text'],
    load_from_cache_file = False,
    desc="Running tokenizer on dataset",
)

block_size = 1024
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop,
    # you can customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=124,
    load_from_cache_file=False,
    desc=f"Grouping texts in chunks of {block_size}",
)

train_dataset = lm_datasets["train"]
eval_dataset = lm_datasets["validation"]

train_dataset.save_to_disk('/content/dataset/train2')
eval_dataset.save_to_disk('/content/dataset/eval2')


procesing dataset
grouping processed dataset


Generating train split: 0 examples [00:00, ? examples/s]

Running tokenizer on dataset (num_proc=32):   0%|          | 0/63 [00:00<?, ? examples/s]

num_proc must be <= 7. Reducing num_proc to 7 for dataset of size 7.


Running tokenizer on dataset (num_proc=7):   0%|          | 0/7 [00:00<?, ? examples/s]

...

Grouping texts in chunks of 1024 (num_proc=63):   0%|          | 0/63 [00:00<?, ? examples/s]

num_proc must be <= 7. Reducing num_proc to 7 for dataset of size 7.


Grouping texts in chunks of 1024 (num_proc=7):   0%|          | 0/7 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/63 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7 [00:00<?, ? examples/s]

In [ ]:
!python /content/ZymCTRL/5.run_clm-post.py --tokenizer_name AI4PD/ZymCTRL \
--do_train --do_eval --output_dir output --overwrite_output_dir true \
--evaluation_strategy steps --eval_steps 10 --logging_steps 5 --save_steps 500 \
--num_train_epochs 20 --per_device_train_batch_size 1 --per_device_eval_batch_size 2 \
--cache_dir '.' --save_total_limit 2 --learning_rate 8e-05 --dataloader_drop_last True \
--model_name_or_path AI4PD/ZymCTRL


2024-09-28 14:19:01.782974: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-28 14:19:01.804680: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-28 14:19:01.811138: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-28 14:19:03.046303: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
09/28/2024 14:

In [ ]:
import torch
from transformers import GPT2LMHeadModel, AutoTokenizer
import os
from tqdm import tqdm
import math

def remove_characters(sequence, char_list):
    "This function removes special tokens used during training."
    columns = sequence.split('<sep>')
    seq = columns[1]
    for char in char_list:
        seq = seq.replace(char, '')
    return seq

def calculatePerplexity(input_ids,model,tokenizer):
    "This function computes perplexities for the generated sequences"
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    loss, logits = outputs[:2]
    return math.exp(loss)

def main(label, model,special_tokens,device,tokenizer):
    # Generating sequences
    input_ids = tokenizer.encode(label,return_tensors='pt').to(device)
    outputs = model.generate(
        input_ids,
        top_k=9, #tbd
        repetition_penalty=1.2,
        max_length=1024,
        eos_token_id=1,
        pad_token_id=0,
           do_sample=True,
           num_return_sequences=20) # Depending non your GPU, you'll be able to generate fewer or more sequences. This runs in an A40.

    # Check sequence sanity, ensure sequences are not-truncated.
    # The model will truncate sequences longer than the specified max_length (1024 above). We want to avoid those sequences.
    new_outputs = [ output for output in outputs if output[-1] == 0]
    if not new_outputs:
        print("not enough sequences with short lengths!!")

    # Compute perplexity for every generated sequence in the batch
    ppls = [(tokenizer.decode(output), calculatePerplexity(output, model, tokenizer)) for output in new_outputs ]

    # Sort the batch by perplexity, the lower the better
    ppls.sort(key=lambda i:i[1]) # duplicated sequences?

    # Final dictionary with the results
    sequences={}
    sequences[label] = [(remove_characters(x[0], special_tokens), x[1]) for x in ppls]

    return sequences

if __name__=='__main__':
    device = torch.device("cuda") # Replace with 'cpu' if you don't have a GPU - but it will be slow
    print('Reading pretrained model and tokenizer')
    tokenizer = AutoTokenizer.from_pretrained('/content/output') # change to ZymCTRL location
    model = GPT2LMHeadModel.from_pretrained('/content/output').to(device) # change to ZymCTRL location
    special_tokens = ['<start>', '<end>', '<|endoftext|>','<pad>',' ', '<sep>']

    # change to the appropriate EC classes
    labels=['3.2.1.1'] # nitrilases. You can put as many labels as you want.

    for label in tqdm(labels):
        # We'll run 100 batches per label. 20 sequences will be generated per batch.
        for i in range(0,100):
            sequences = main(label, model, special_tokens, device, tokenizer)
            for key,value in sequences.items():
                for index, val in enumerate(value):
                    fn = open(f"/content/drive/MyDrive/finetune_out2/{label}_{i}_{index}.fasta", "w")
                    fn.write(f'>{label}_{i}_{index}\t{val[1]}\n{val[0]}')
                    fn.close()


Reading pretrained model and tokenizer


  0%|          | 0/1 [00:00<?, ?it/s]

not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short lengths!!
not enough sequences with short le

100%|██████████| 1/1 [18:15<00:00, 1095.68s/it]


## Rank the FASTA files based on perplexity
After generating the sequences, we can rank them based on their perplexity, calculated using ZymCTRL, where a lower perplexity is better. We can generate 10,000 sequences and set a threshold for perplexity to select the better sequences. Unfortunately, despite using Colab Pro, I quickly ran out of computational resources, which limited my ability to run the model multiple times and generate more sequences.

In [ ]:
import os
from Bio import SeqIO

fasta_dir = '/content/drive/MyDrive/finetune_out2'

fasta_data = []

# Loop through each FASTA file in the directory
for file_name in os.listdir(fasta_dir):
    if file_name.endswith(".fasta"):
        file_path = os.path.join(fasta_dir, file_name)
        with open(file_path, 'r') as file:
            # Parse the FASTA file using SeqIO
            for record in SeqIO.parse(file, "fasta"):
                header = record.description
                id_perplexity = header.split('\t')

                if len(id_perplexity) == 2:  # Ensure we correctly extract ID and perplexity
                    sequence_id = id_perplexity[0]
                    perplexity = float(id_perplexity[1])  # Extract the perplexity as a float

                    # Append the perplexity and sequence to the data list
                    fasta_data.append((perplexity, record.seq, sequence_id, file_name))

fasta_data_sorted = sorted(fasta_data, key=lambda x: x[0])


for rank, (perplexity, sequence, sequence_id, file_name) in enumerate(fasta_data_sorted, start=1):
    new_id = f"Rank_{rank}"
    new_description = f"{new_id}\t{perplexity}"

    # Open a new file for writing (this will overwrite or create new files)
    output_file = os.path.join(fasta_dir, f"{new_id}.fasta")
    with open(output_file, 'w') as output_handle:
        output_handle.write(f">{new_description}\n")
        output_handle.write(str(sequence) + "\n")

print("FASTA files have been processed, ranked, and saved without extra spaces in the sequences.")

FASTA files have been processed, ranked, and saved without extra spaces in the sequences.


Just ignore this part, it’s for my own use. :)

In [ ]:
def add_sequence_labels(text):

    paragraphs = text.strip().split('\n')

    result = []

    # Iterate over paragraphs and add >sp|SEQ(number) before each one
    for i, paragraph in enumerate(paragraphs):
        seq_label = f">sp|SEQ{i + 1}\n"
        result.append(seq_label + paragraph)

    return "\n".join(result)



text = """
ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVKEGNQGDKSMSNWYWLYQPTSYQIGNRYLGTEQEFKEMCAAAEEYGIKVIVDAVLNHTTSDYAAISNEVKSIPNWTHGNTPIKNWSDRWDVTQHSLLGLYDWNTQNTQVQSYLKRFLDRALNDGADGFRFDAAKHIELPDDGSYGSQFWPNITNTSAEFQYGEILQDSVSRDAAYANYMDVTASNYGHSIRSALKNRNLGVSNISHYAIDVSADKLVTWVESHDTYANDDEESTWMSDDDIRLGWAVIASRSGSTPLFFSRPEGGGNGVRFPGKSQIGDRGSALFEDQAITAVNRFHNVMAGQPEELSNPNGNNQIFMNQRGSHGVVLANAGSSSVSINTATKLPDGRYDNKAGAGSFQVNDGKLTGTINARSVAVLYAD
ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVKEGNQGDKSMSNWYWLYQPTSYHIGNRYLGTEQEFKEMCAAAEEYGIKVIVDAVLNHTTSDYAAISNEVKSIPNWTHGNTPIKNWSDRWDVTQHSLLGLYDWNTQNTQVQSYLKRFLDRALNDGADGFRFDAAKHIELPDDGSYGSQFWPNITNTSAEFQYGEILQDSVSRDAAYANYMDITASNYGHSIRSALKNRNLGVSNISHYAIDVSADKLVTWVESHDTYANDDEESTWMSDDDIRLGWAVIASRSGSTPLFFSRPEGGGNGVRFPGKSQIGDRGSALFEDQAITAVNRFHNVMAGQPEELSNPNGNNQIFMNQRGSHGVVLANAGSSSVSINTATKLPDGRYDNKAGAGSFQVNDGKLTGTINARSVAVLYPD
ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVKEGNQGDKSMSNWYWLYQPTSYQIGNRYLGTEQEFKEMCAAAEEYGIKVIVDAVLNHTTSDYAAISNEVKSIPNWTHGNTPIKNWSDRWDVTQNSLLGLYDWNTQNTQVQSYLKRFLDRALNDGADGFRFDAAKHIELPDDGSYGSQFWPNITNTSAEFQYGEILQDSVSRDAAYANYMDVTASNYGHSIRSALKNRNLGVSNISHYAIDVSADKLVTWVESHDTYANDDEESTWMSDDDIRLGWAVIASRSGSTPLFFSRPEGGGNGVRFPGKSQIGDRGSALFEDQAITAVNRFHNVMAGQPEELSNPNGNNQIFMNQRGSHGVVLANAGSSSVSINTATKLPDGRYDNKAGAGSFQVNDGKLTGTINARSVAVLYPD
"""

modified_text = add_sequence_labels(text)

print(modified_text)


>sp|SEQ1
ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVKEGNQGDKSMSNWYWLYQPTSYQIGNRYLGTEQEFKEMCAAAEEYGIKVIVDAVLNHTTSDYAAISNEVKSIPNWTHGNTPIKNWSDRWDVTQHSLLGLYDWNTQNTQVQSYLKRFLDRALNDGADGFRFDAAKHIELPDDGSYGSQFWPNITNTSAEFQYGEILQDSVSRDAAYANYMDVTASNYGHSIRSALKNRNLGVSNISHYAIDVSADKLVTWVESHDTYANDDEESTWMSDDDIRLGWAVIASRSGSTPLFFSRPEGGGNGVRFPGKSQIGDRGSALFEDQAITAVNRFHNVMAGQPEELSNPNGNNQIFMNQRGSHGVVLANAGSSSVSINTATKLPDGRYDNKAGAGSFQVNDGKLTGTINARSVAVLYAD
>sp|SEQ2
ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVKEGNQGDKSMSNWYWLYQPTSYHIGNRYLGTEQEFKEMCAAAEEYGIKVIVDAVLNHTTSDYAAISNEVKSIPNWTHGNTPIKNWSDRWDVTQHSLLGLYDWNTQNTQVQSYLKRFLDRALNDGADGFRFDAAKHIELPDDGSYGSQFWPNITNTSAEFQYGEILQDSVSRDAAYANYMDITASNYGHSIRSALKNRNLGVSNISHYAIDVSADKLVTWVESHDTYANDDEESTWMSDDDIRLGWAVIASRSGSTPLFFSRPEGGGNGVRFPGKSQIGDRGSALFEDQAITAVNRFHNVMAGQPEELSNPNGNNQIFMNQRGSHGVVLANAGSSSVSINTATKLPDGRYDNKAGAGSFQVNDGKLTGTINARSVAVLYPD
>sp|SEQ3
ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVKEGNQGDKSMSNWYWLYQPTSYQIGNRYLGTEQEFKEMCAAAEEYGIKVIVDAVLNHTTSDYAAISNEVKSIPNWT

## 2. Using ESM with added noise to unconserved positions

I obtained the per-residue representation of each sequence, added Gaussian noise to the non-conserved positions, and then mapped the modified representation back to the sequence space. This approach is inspired by the [PePerCLIP](https://www.biorxiv.org/content/10.1101/2023.06.26.546591v2) model, which is used for peptide design.

In [17]:
!pip install torch
!pip install fair-esm
!sudo apt-get install clustalo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 2.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libargtable2-0
The following NEW packages will be installed:
  clustalo libargtable2-0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 273 kB of archives.
After this operation, 694 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libargtable2-0 amd64 13-1.1 [14.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 clustalo amd64 1.2.4-7 [259 kB]
Fetched 273 kB in 1s (404 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend:

### Finding non-conserved positions
So, we first need to find the non-conserved positions in the sequences, then add noise to the representations of those amino acids, and finally bring them back to the sequence space. Here, I want to identify the non-conserved positions.

In [ ]:
from Bio.Align import MultipleSeqAlignment
from Bio.Align import AlignInfo
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import AlignIO
import pandas as pd

file_path = '/content/drive/MyDrive/filtered_sequences.csv'
sequences_df = pd.read_csv(file_path)

sequences = sequences_df['mutated_sequence'].dropna().tolist()

seq_records = [SeqRecord(Seq(seq), id=str(i)) for i, seq in enumerate(sequences)]

# Perform multiple sequence alignment
from Bio.Align.Applications import ClustalOmegaCommandline
with open("input_sequences.fasta", "w") as fasta_file:
    for record in seq_records:
        fasta_file.write(f">{record.id}\n{record.seq}\n")

clustalomega_cline = ClustalOmegaCommandline(infile="input_sequences.fasta", outfile="aligned.fasta", verbose=True, auto=True)
clustalomega_cline()

alignment = AlignIO.read("aligned.fasta", "fasta")

# Identify non-conserved positions
non_conserved_positions = []
for i in range(len(alignment[0])):
    column = [record.seq[i] for record in alignment]
    if len(set(column)) > 1:  # Check if there's more than one amino acid type at this position
        non_conserved_positions.append(i)

print("Non-conserved positions:", non_conserved_positions)


/usr/local/lib/python3.10/dist-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


Non-conserved positions: [42, 67, 80, 87, 99, 108, 118, 130, 138, 141, 144, 187, 190, 191, 225, 232, 246, 253, 275, 357, 401, 406, 423]


### Performing ESM
I applied ESM by adding noise to the sequences and mapping them back to sequence space. I used the first sequence, which had the highest activity, as the reference sequence.

In [ ]:
import torch
import esm
import numpy as np
import pandas as pd

model_name = 'esm2_t6_8M_UR50D'  # Smaller model for demonstration
model, alphabet = esm.pretrained.load_model_and_alphabet(model_name)
model.eval()  # Set the model to evaluation mode

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


sequence = "ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVKEGNQGDKSMSNWYWLYQPTSYQIGNRYLGTEQEFKEMCAAAEEYGIKVIVDAVLNHTTSDYAAISNEVKSIPNWTHGNTPIKNWSDRWDVTQHSLLGLYDWNTQNTQVQSYLKRFLDRALNDGADGFRFDAAKHIELPDDGSYGSQFWPNITNTSAEFQYGEILQDSVSRDAAYANYMDVTASNYGHSIRSALKNRNLGVSNISHYAIDVSADKLVTWVESHDTYANDDEESTWMSDDDIRLGWAVIASRSGSTPLFFSRPEGGGNGVRFPGKSQIGDRGSALFEDQAITAVNRFHNVMAGQPEELSNPNGNNQIFMNQRGSHGVVLANAGSSSVSINTATKLPDGRYDNKAGAGSFQVNDGKLTGTINARSVAVLYAD"  # Replace with your sequence
sequence = sequence.upper()

batch_converter = alphabet.get_batch_converter()
data = [("protein1", sequence)]
batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_tokens = batch_tokens.to(device)

with torch.no_grad():
    # Obtain the model's representations for the sequence
    results = model(batch_tokens, repr_layers=[model.num_layers], return_contacts=False)
    token_embeddings = results["representations"][model.num_layers]

token_embeddings = token_embeddings[0, 1:-1]  # Shape: (L, D)

# 6. Generate N sequences with noise added to specific positions
N = 2000  # Number of sequences to generate
k = 0.25  # Scaling factor for noise, set it small to avoid large changes

L, D = token_embeddings.shape

# Expand token embeddings to shape (N, L, D)
token_embeddings_expanded = token_embeddings.unsqueeze(0).expand(N, L, D)

noise = torch.zeros((N, L, D), device=device)

# Add noise only to specific positions
for pos in non_conserved_positions:
    noise[:, pos, :] = torch.randn((N, D), device=device)

perturbed_embeddings = token_embeddings_expanded + k * noise

# Map perturbed embeddings back to amino acid probabilities
final_layer = model.lm_head  # Linear layer: (embedding_dim) -> (alphabet_size)

logits = final_layer(perturbed_embeddings)  # Shape: (N, L, alphabet_size)

probabilities = torch.softmax(logits, dim=-1)  # Shape: (N, L, alphabet_size)

predicted_tokens = torch.argmax(probabilities, dim=-1)  # Shape: (N, L)

# Convert token indices back to amino acids
tokens = alphabet.tok_to_idx
idx_to_token = {idx: tok for tok, idx in tokens.items()}

amino_acids = set(alphabet.standard_toks)
original_first_amino_acid = sequence[0]

sequences = []
for i in range(N):
    seq_tokens = predicted_tokens[i]  # Shape: (L,)
    new_sequence = ''.join([idx_to_token[int(token)] for token in seq_tokens])
    new_sequence = ''.join([aa if aa in amino_acids else '' for aa in new_sequence])

    new_sequence = original_first_amino_acid + new_sequence[1:]

    sequences.append(new_sequence)

sequences_df = pd.DataFrame(sequences, columns=["Generated Sequence"])
sequences_df = sequences_df.drop_duplicates(subset=['Generated Sequence'])
sequences_df.to_csv('/content/drive/My Drive/esm_generated_seq.csv', index=False)
print (f'number of sequences generqted is', sequences_df.shape[0])

print("Generated sequences saved to: /content/drive/My Drive/esm_generated_seq.csv")


number of sequences generqted is 574
Generated sequences saved to: /content/drive/My Drive/esm_generated_seq.csv


## Check if generated sequences Are Nature-Like
It is not necessary to run this block, as almost all of the sequences are nature-like, and the process is both time-consuming and computationally expensive. I only used it the first time to ensure accuracy when I initially generated the sequences.

In [ ]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import time

# Function to BLAST a protein sequence
def blast_sequence(sequence):

    result_handle = NCBIWWW.qblast("blastp", "nr", sequence)  # 'nr' is the non-redundant protein sequence database

    # Parse the BLAST output
    blast_records = NCBIXML.read(result_handle)

    return blast_records

# Function to check if the sequence is similar to any known protein
def check_nature_like(blast_record):
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < 0.01:  # E-value threshold
                return True
    return False

csv_file = '/content/generated_sequences.csv'
df = pd.read_csv(csv_file)

sequences = df.iloc[:, 0].tolist()  t

# Check if sequences are nature-like with indexing
nature_like_sequences = []
for index, seq in enumerate(sequences, start=1):
    print(f"Blasting sequence number {index}...")
    blast_result = blast_sequence(seq)
    is_nature_like = check_nature_like(blast_result)

    if is_nature_like:
        print(f"Sequence number {index} is nature-like.")
        nature_like_sequences.append(seq)
    else:
        print(f"Sequence number {index} is not nature-like.")

    time.sleep(5)  # Delay to avoid overwhelming NCBI's servers

print(f"Total nature-like sequences: {len(nature_like_sequences)}")


Blasting sequence number 1...
Sequence number 1 is nature-like.
Blasting sequence number 2...
Sequence number 2 is nature-like.
Blasting sequence number 3...
Sequence number 3 is nature-like.
Blasting sequence number 4...
Sequence number 4 is nature-like.
Blasting sequence number 5...
Sequence number 5 is nature-like.
Blasting sequence number 6...
Sequence number 6 is nature-like.
Blasting sequence number 7...
Sequence number 7 is nature-like.
Blasting sequence number 8...
Sequence number 8 is nature-like.
Blasting sequence number 9...
Sequence number 9 is nature-like.
Blasting sequence number 10...
Sequence number 10 is nature-like.
Total nature-like sequences: 10


# Scoring the sequencese
## 1. Using PCA

I’m looking to make a CSV file from the FASTA files generated by ZymCTRL.

In [6]:

import pandas as pd
from Bio import SeqIO
import os

Path = "/content/drive/MyDrive/finetune_out"


sequence_data = []

# Iterate through the extracted FASTA files
for filename in sorted(os.listdir(Path)):
    if filename.endswith(".fasta"):
        filepath = os.path.join(Path, filename)

        # Parse the FASTA file using BioPython
        for record in SeqIO.parse(filepath, "fasta"):
            sequence_data.append([record.id, str(record.seq)])

df = pd.DataFrame(sequence_data, columns=["ID", "mutated_sequence"])
csv_output_path = "/content/drive/MyDrive/finetune_out/sequences.csv"
df.to_csv(csv_output_path, index=False)

print(f"FASTA data saved to {csv_output_path}")


FASTA data saved to /content/drive/MyDrive/finetune_out/sequences.csv


Now I need to merge two csv files (the cleaned data and generated sequences.)

In [14]:
import pandas as pd
import numpy as np

file_path_1 = '/content/drive/MyDrive/filtered_sequences.csv'
file_path_2 = '/content/drive/MyDrive/finetune_out/sequences.csv'


data_1 = pd.read_csv(file_path_1)

data_2 = pd.read_csv(file_path_2)

data_1['ID'] = data_1['ID'].astype(str)
data_2['ID'] = data_2['ID'].astype(str)

data_2['activity_dp7'] = np.nan

combined_data = pd.concat([data_1, data_2], ignore_index=True)

combined_data.to_csv('/content/drive/MyDrive/merged_file.csv', index=False)

print(combined_data.head())


  ID                                   mutated_sequence  activity_dp7
0  1  ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVK...     10.552655
1  2  ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVK...      9.338325
2  3  ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVK...      8.289414
3  4  ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVK...      7.007689
4  5  ATAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPIMQVK...      6.898567


In this step I need to find the representation of each sequence to apply PCA on them. I choose esm to find the representation.

In [18]:
import pandas as pd
import torch
import esm

csv_file_path = "/content/drive/MyDrive/merged_file.csv"
sequences_df = pd.read_csv(csv_file_path)

model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()

sequences = sequences_df['mutated_sequence'].tolist()

batch_size = 10
sequence_embeddings = []

for i in range(0, len(sequences), batch_size):
    batch_seqs = sequences[i:i + batch_size]
    sequence_tuples = [(str(idx), seq) for idx, seq in enumerate(batch_seqs)]

    batch_labels, batch_strs, batch_tokens = batch_converter(sequence_tuples)

    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[6], return_contacts=False)
        token_representations = results["representations"][6]


    for j, (label, seq) in enumerate(sequence_tuples):
        representation = token_representations[j, 1: len(seq) + 1].mean(0)
        sequence_embeddings.append(representation.cpu().numpy())

sequences_df['esm_representation'] = sequence_embeddings

output_file_path = "/content/drive/MyDrive/esm_representations.csv"
sequences_df.to_csv(output_file_path, index=False)


Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t6_8M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D-contact-regression.pt


To reduce the dimensionality of the esm representations using PCA, I first need to determine how many principal components to retain. I calculate the variance explained by each component to ensure we preserve enough information. This approach helps identify the number of components needed to retain 99% of the data’s variance.

In [19]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import ast

file_path = '/content/drive/MyDrive/esm_representations.csv'
data = pd.read_csv(file_path)

# Clean the 'esm_representation' column by removing brackets and unnecessary characters
def clean_representation_string(rep_str):
    rep_str_cleaned = rep_str.replace("[", "").replace("]", "").replace("\n", " ").replace("  ", " ").strip()
    return np.array([float(x) for x in rep_str_cleaned.split()])

data['esm_representation'] = data['esm_representation'].apply(clean_representation_string)

esm_matrix = np.stack(data['esm_representation'].values)

pca = PCA().fit(esm_matrix)

# Get the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Cumulative explained variance
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# Find the number of components that explain 99% of the variance
num_components_99 = np.argmax(cumulative_explained_variance >= 0.99) + 1

print(f'Number of components explaining 99% variance: {num_components_99}')
print(f'Cumulative explained variance: {cumulative_explained_variance}')


Number of components explaining 99% variance: 22
Cumulative explained variance: [0.38939513 0.53247843 0.6466768  0.7131372  0.76372909 0.80459686
 0.83848496 0.86662532 0.89287509 0.91153634 0.929406   0.94372637
 0.95340051 0.96175455 0.96888227 0.97509639 0.97953362 0.98277881
 0.98554166 0.98812402 0.98988574 0.99154884 0.99298103 0.99415396
 0.99526794 0.99620989 0.99702216 0.99761168 0.99806046 0.99840383
 0.99868343 0.99893295 0.99915303 0.99931635 0.99946737 0.99957008
 0.99963576 0.99969175 0.99973691 0.99977702 0.99981377 0.99984338
 0.99986443 0.99988025 0.99989443 0.9999062  0.99991604 0.99992446
 0.99993203 0.99993856 0.99994428 0.99994952 0.99995408 0.99995849
 0.99996267 0.9999664  0.9999698  0.99997262 0.99997488 0.99997688
 0.99997876 0.99998045 0.999982   0.99998344 0.99998467 0.99998586
 0.99998689 0.99998786 0.99998879 0.99998969 0.99999054 0.99999131
 0.99999201 0.99999268 0.99999331 0.99999388 0.99999442 0.99999491
 0.99999533 0.99999574 0.99999612 0.99999645 0.99

In this step I can perform PCA to reduce the dimension. in the previous step we found the Number of components explaining 99% variance is 22.

In [20]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA


file_path = '/content/drive/MyDrive/esm_representations.csv'
data = pd.read_csv(file_path)

esm_embeddings = data['esm_representation']

# Function to parse the string representation of the esm embedding
def parse_embedding(embedding_str):
    embedding_values = embedding_str.strip('[]').split()
    embedding = np.array([float(val) for val in embedding_values])
    return embedding

embeddings = esm_embeddings.apply(parse_embedding)
embedding_matrix = np.vstack(embeddings.values)

# Perform PCA to reduce the dimensions to the top components
pca = PCA(n_components = num_components_99)
principal_components = pca.fit_transform(embedding_matrix)

pca_representation = pd.Series([list(pc) for pc in principal_components], name="PCA_representation")
final_df = data.copy()
final_df['PCA_representation'] = pca_representation

output_file = '/content/drive/MyDrive/PCA_embeddings.csv'
final_df.to_csv(output_file, index=False)

output_file


'/content/drive/MyDrive/PCA_embeddings.csv'

Here I used Gaussian Process Regression (GPR) to label the unlabeled data, given that the number of labeled samples is small (130) and the dimensionality of the feature space is high (22). GPR is an ideal model in this scenario because it works well with small datasets and provides uncertainty estimates along with predictions. In this step, GPR is applied to predict the missing labels (activity) for the unlabeled data, and the results are ranked based on these predicted values.

In [25]:
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from ast import literal_eval

file_path = '/content/drive/MyDrive/PCA_embeddings.csv'
data = pd.read_csv(file_path)

data['PCA_representation'] = data['PCA_representation'].apply(literal_eval)

# Separate labeled and unlabeled data
labeled_data = data[~data['activity_dp7'].isna()]  # Data with known activity
unlabeled_data = data[data['activity_dp7'].isna()]  # Data without activity

X_train = np.vstack(labeled_data['PCA_representation'].values)
y_train = labeled_data['activity_dp7'].values

# For unlabeled data, extract only the PCA features
X_unlabeled = np.vstack(unlabeled_data['PCA_representation'].values)

# Define a kernel for the Gaussian Process
kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))

gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

gp.fit(X_train, y_train)

predicted_activity, sigma = gp.predict(X_unlabeled, return_std=True)

unlabeled_data.loc[:, 'predicted_activity_dp7'] = predicted_activity

# Combine the labeled and unlabeled data into one dataset
combined_data = pd.concat([labeled_data, unlabeled_data], ignore_index=True)

combined_data = combined_data.drop(columns=['activity_dp7'])

sorted_data = combined_data.sort_values(by='predicted_activity_dp7', ascending=False)

sorted_file_path = '/content/drive/MyDrive/sorted_by_activity.csv'
sorted_data.to_csv(sorted_file_path, index=False)

# Extract the top 100 sequences based on predicted_activity_dp7
top_100_sequences = sorted_data[['mutated_sequence', 'predicted_activity_dp7']].head(100)

top_100_file_path = '/content/drive/MyDrive/top_100_sequences.csv'
top_100_sequences.to_csv(top_100_file_path, index=False)

print(f"Data sorted by 'predicted_activity_dp7', 'activity_dp7' column removed, and saved to {sorted_file_path}")
print(f"Top 100 sequences saved to {top_100_file_path}")


Data sorted by 'predicted_activity_dp7', 'activity_dp7' column removed, and saved to /content/drive/MyDrive/sorted_by_activity.csv
Top 100 sequences saved to /content/drive/MyDrive/top_100_sequences.csv


<ipython-input-25-c01ff142e6c2>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unlabeled_data.loc[:, 'predicted_activity_dp7'] = predicted_activity


I did the same with the data obtained using the second approach (ESM with noise added to the unconserved residues), and the results were embarrassing, so I prefer not to publish it 😓

## 2. Using MERGE

MERGE represents a method that combines direct coupling analysis and machine learning techniques to predict a protein's fitness from sequence.

In [ ]:
!git clone https://github.com/amillig/MERGE.git

fatal: destination path 'MERGE' already exists and is not an empty directory.


You can find MERGE instruction in their github [page](https://github.com/amillig/MERGE).  Unfortunately, MERGE didn't work because the variant-fitness pairs require tracking the changes one by one, which is impossible for my dataset. It also requires to have Uniref100.fasta (over 100Gb) to run and generate an MSA (it is possible to generate MSA with online platforms or AlphaFold) so I prefer to use another method. But hte author claims the fitness landscape can be explored to generate improved variants applying Metropolis Hastings algorithm. 

## 3. Training an encoder, decoder
Even though I used Google Colab Pro, I didn’t have enough computational resources to generate sufficient data with ZymCTRL, so training or fine-tuning an encoder-decoder model won’t be possible.

## 4. Learning the Sequence of Variant-Fitness Pairs
I am trying to learn about the changes that improve activity. I start with the sequence that has an activity of 0 and then move to the next sequence to check the changes. I then tokenize these changes. For example, from the sequence with activity 0 to the next one, the changes are W81K and L100M, which I represent as A and B, respectively. The opposite direction will be -A and -B. I did this for all the sequences.

In [ ]:
import pandas as pd
import string

file_path = '/content/filtered_sequences_removed_missing.csv'
df = pd.read_csv(file_path)

df_sorted = df.sort_values(by='activity_dp7').reset_index(drop=True)

# Function to find the differences between two sequences
def find_sequence_differences(seq1, seq2):
    differences = []
    for i, (a, b) in enumerate(zip(seq1, seq2)):
        if a != b:
            differences.append(f"{a}{i+1}{b}")
    return ', '.join(differences) if differences else 'No difference'

# Create a new column for the differences between consecutive sequences
df_sorted['changes'] = ['No previous sequence'] + [
    find_sequence_differences(df_sorted.loc[i, 'mutated_sequence'], df_sorted.loc[i-1, 'mutated_sequence'])
    for i in range(1, len(df_sorted))
]

change_to_token = {}
alphabet = string.ascii_uppercase  # A-Z letters for tokens

# Function to generate tokens for changes
def generate_tokens(changes):
    token_counter = 0
    for change in changes:
        # Get the forward (e.g., W81K) and reverse (e.g., K81W) form of the change
        forward_change = change
        reverse_change = f"{change[-1]}{change[1:-1]}{change[0]}"

        if forward_change not in change_to_token:
            token = alphabet[token_counter % len(alphabet)]
            change_to_token[forward_change] = token
            change_to_token[reverse_change] = f"-{token}"
            token_counter += 1

# Extract all the unique changes from the 'changes' column
all_changes = set()
for change_string in df_sorted['changes']:
    individual_changes = change_string.split(', ')
    all_changes.update(individual_changes)

generate_tokens(all_changes)

# Function to map changes to the specified tokens and ensure no row starts with '='
def map_changes_to_tokens(change_string):

    individual_changes = change_string.split(', ')
    mapped_changes = [change_to_token.get(change, change) for change in individual_changes]
    tokenized_string = ', '.join(mapped_changes)

    if tokenized_string.startswith('='):
        tokenized_string = "'" + tokenized_string

    return tokenized_string

df_sorted['tokenized_changes'] = df_sorted['changes'].apply(map_changes_to_tokens)


output_file = '/content/filtered_sequences_with_tokenized_changes.csv'
df_sorted.to_csv(output_file, index=False)

print(f"Conversion complete! The updated file has been saved as {output_file}")


Conversion complete! The updated file has been saved as /content/filtered_sequences_with_tokenized_changes.csv


To check how many tokens we have in total.

In [ ]:
import pandas as pd

file_path = '/content/filtered_sequences_with_tokenized_changes.csv'
df = pd.read_csv(file_path)

all_tokens = df['tokenized_changes'].str.split(', ').explode().unique()

num_unique_tokens = len(all_tokens)

# Output the number of unique tokens
print("Total number of unique tokens:", num_unique_tokens)


Total number of unique tokens: 50


Now I want to check if any tokens are duplicated in any rows or if there are reverse duplicates.

In [ ]:
import pandas as pd

file_path = '/content/filtered_sequences_with_tokenized_changes.csv'
df = pd.read_csv(file_path)

# Function to check if two rows are reverse duplicates
def is_reverse_duplicate(row1, row2):
    tokens1 = row1.split(', ')
    tokens2 = row2.split(', ')

    # They should have the same number of tokens
    if len(tokens1) != len(tokens2):
        return False

    # Check if each token in tokens1 has the exact reverse in tokens2
    for t1, t2 in zip(tokens1, tokens2):
        if t1 != f"-{t2}" and t2 != f"-{t1}":
            return False
    return True

exact_duplicates = []
reversed_duplicates = []

# Step 1: Check for exact duplicates
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        if df.loc[i, 'tokenized_changes'] == df.loc[j, 'tokenized_changes']:
            exact_duplicates.append((df.loc[i, 'ID'], df.loc[j, 'ID']))

# Step 2: Check for reverse duplicates
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        if is_reverse_duplicate(df.loc[i, 'tokenized_changes'], df.loc[j, 'tokenized_changes']):
            reversed_duplicates.append((df.loc[i, 'ID'], df.loc[j, 'ID']))

# Display the results
if exact_duplicates:
    print("Exact duplicate rows found:")
    for dup in exact_duplicates:
        print(f"Row {dup[0]} is an exact duplicate of row {dup[1]}")
else:
    print("No exact duplicate rows found.")

if reversed_duplicates:
    print("Reversed duplicate rows found:")
    for dup in reversed_duplicates:
        print(f"Row {dup[0]} is a reverse duplicate of row {dup[1]}")
else:
    print("No reversed duplicate rows found.")


Exact duplicate rows found:
Row 64 is an exact duplicate of row 26
Row 22 is an exact duplicate of row 12
Reversed duplicate rows found:
Row 37 is a reverse duplicate of row 15


 ##### As you can see, Row 37 is a reverse duplicate of Row 15. This suggests that certain changes enhance activity in one sequence, while their reverse boosts activity in another. I didn’t expect this, and I’m really excited to discover it :) it makes the problem even more mysterious! 

 The number of tokens I found is 50 and the number of data points is 130, which makes it impossible to learn the changes.

#END